In [26]:
import cv2
import numpy as np
import os
from tqdm import tqdm
from scipy.signal import savgol_filter
import torch
import torchvision.models as models
from torchvision import transforms as T

# --------------------- Lightweight Segmentation ---------------------
class DynamicMaskGenerator:
    def __init__(self):
        self.model = models.segmentation.deeplabv3_mobilenet_v3_large(pretrained=True)
        self.model = self.model.to('cpu').eval()
        self.transform = T.Compose([
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
    def generate_mask(self, frame):
        input_tensor = self.transform(frame).unsqueeze(0).to('cpu')
        with torch.no_grad():
            output = self.model(input_tensor)['out'][0]
        output = output.argmax(0).cpu().numpy()
        # Adjust these indices based on your model's class mapping
        dynamic_mask = np.isin(output, [1, 2, 3])  # Example: people, vehicles, plants
        return dynamic_mask.astype(np.uint8) * 255

# --------------------- Robust Feature Handling ---------------------
class FeatureManager:
    def __init__(self):
        self.detectors = {
            'fast': cv2.FastFeatureDetector_create(threshold=50),
            'gftt': cv2.GFTTDetector_create(maxCorners=800, qualityLevel=0.02),
            'orb': cv2.ORB_create(nfeatures=1500, fastThreshold=20)
        }
        self.matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)

    def detect_features(self, gray):
        features = []
        for name, det in self.detectors.items():
            kp = det.detect(gray, None)
            if name == 'orb':
                kp, des = det.compute(gray, kp)
            else:
                des = None
            features.append((kp, des))
        return features

    def match_features(self, des1, des2, ratio=0.75):
        if des1 is None or des2 is None or len(des1) < 2 or len(des2) < 2:
            return []
        matches = self.matcher.knnMatch(des1, des2, k=2)
        return [m[0] for m in matches if len(m) == 2 and m[0].distance < ratio * m[1].distance]

# --------------------- CPU-Optimized Stabilization ---------------------
class AdvancedStabilizer:
    def __init__(self, frames_path, output_path, fps=30):
        self.frames_path = frames_path
        self.output_path = output_path
        self.fps = fps
        self.mask_generator = DynamicMaskGenerator()
        self.feature_mgr = FeatureManager()
        
        # Load frames with numeric sorting
        self.frame_files = sorted(
            [os.path.join(frames_path, f) for f in os.listdir(frames_path) 
             if f.lower().endswith(('.png','.jpg','.jpeg'))],
            key=lambda x: int(''.join(filter(str.isdigit, os.path.basename(x)))) if ''.join(filter(str.isdigit, os.path.basename(x))) else 0
        )
        
        if not self.frame_files:
            raise ValueError("No valid image files found")

    def process_frame(self, idx):
        frame = cv2.imread(self.frame_files[idx])
        if frame is None:
            return None, None, None
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        mask = self.mask_generator.generate_mask(frame)
        return gray, mask, frame

    def compute_transform(self, prev_gray, curr_gray, mask):
        prev_feats = self.feature_mgr.detect_features(prev_gray)
        curr_feats = self.feature_mgr.detect_features(curr_gray)
        
        matches = []
        for (kp1, des1), (kp2, des2) in zip(prev_feats, curr_feats):
            if des1 is not None and des2 is not None:
                matches.extend(self.feature_mgr.match_features(des1, des2))
        
        if len(matches) > 4:
            pts1 = np.float32([prev_feats[0][0][m.queryIdx].pt for m in matches])
            pts2 = np.float32([curr_feats[0][0][m.trainIdx].pt for m in matches])
            
            # Enhanced validation with border check
            h, w = mask.shape
            valid = [
                mask[int(y), int(x)] == 0 and
                20 < x < w-20 and 20 < y < h-20
                for x, y in pts1
            ]
            pts1 = pts1[valid]
            pts2 = pts2[valid]
            
            if len(pts1) >= 4:
                H, _ = cv2.estimateAffinePartial2D(pts1, pts2, method=cv2.RANSAC, 
                                                 ransacReprojThreshold=4.0)
                return H if H is not None else np.eye(3)[:2]
        return np.eye(3)[:2]

    def stabilize(self):
        # Sequential frame loading and preprocessing (NO MULTIPROCESSING)
        print("Loading and preprocessing frames...")
        results = []
        for i in tqdm(range(len(self.frame_files))):
            result = self.process_frame(i)
            results.append(result)
        
        valid_results = [r for r in results if r[0] is not None]
        if not valid_results:
            raise ValueError("No frames could be processed")
            
        self.gray_frames, self.masks, self.frames = zip(*valid_results)
        
        # Compute motion transforms
        print("Computing transforms...")
        transforms = [np.eye(3)[:2]]
        for i in tqdm(range(1, len(self.gray_frames))):
            H = self.compute_transform(self.gray_frames[i-1], self.gray_frames[i], self.masks[i])
            transforms.append(H)
        
        # Smoothing with Savitzky-Golay filter with proper window handling
        print("Optimizing trajectory...")
        params = np.array([t.flatten() for t in transforms])
        n_frames = params.shape[0]
        
        # Dynamic window length handling
        window_length = min(15, n_frames)
        if window_length % 2 == 0:  # Must be odd
            window_length = max(3, window_length - 1)
        if window_length < 3:
            window_length = 3
        
        smoothed_params = savgol_filter(params.T, window_length=window_length, polyorder=2, axis=1).T
        
        # Prepare stabilized video
        print("Writing output...")
        h, w = self.frames[0].shape[:2]
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(self.output_path, fourcc, self.fps, (w, h))
        
        if not out.isOpened():
            raise RuntimeError("Could not initialize video writer")
        
        for i in tqdm(range(len(self.frames))):
            if i < len(smoothed_params):
                try:
                    # Reshape flattened parameters to 2x3 matrix
                    H_flat = smoothed_params[i]
                    H = H_flat.reshape(2, 3)
                    stabilized = cv2.warpAffine(self.frames[i], H, (w, h), 
                                              borderMode=cv2.BORDER_REFLECT)
                except Exception as e:
                    print(f"Error warping frame {i}: {e}")
                    stabilized = self.frames[i]
            else:
                stabilized = self.frames[i]
            out.write(stabilized)
        
        out.release()
        print(f"Stabilized video saved to: {self.output_path}")

# --------------------- Usage ---------------------
if __name__ == "__main__":
    stabilizer = AdvancedStabilizer(
        frames_path=r"C:\Users\simar\OneDrive\Desktop\Python_stabilization\unzipped_video",
        output_path=r"C:\Users\simar\OneDrive\Desktop\Python_stabilization\cpu_optimized.mp4",
        fps=29.97
    )
    stabilizer.stabilize()


Loading and preprocessing frames...


 55%|█████▌    | 3958/7169 [6:54:47<5:36:30,  6.29s/it]


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\alloc.cpp:73: error: (-4:Insufficient memory) Failed to allocate 2764800 bytes in function 'cv::OutOfMemoryError'
